# Upload dataset

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving dataset.zip to dataset.zip


In [ ]:
!unzip dataset.zip

Archive:  dataset.zip
  inflating: web-BerkStan.mtx        


# Import Libraries and constants

In [ ]:
import numpy as np
from scipy.io import mmread
import networkx as nx

alphas = [0.1, 0.85, 0.95]

# Loading the file

In [ ]:
sparse_matrix = mmread('web-BerkStan.mtx')
csr_matrix = sparse_matrix.tocsr()
subset = csr_matrix[:1000, :1000]
matrix = subset.toarray()
print(matrix.shape)

(1000, 1000)


# Question 1

# Power iteration implementation

In [ ]:
def compute_pagerank(adjacency_matrix, alpha=0.85, tolerance=1e-6, max_iterations=100):
    num_nodes = adjacency_matrix.shape[0]
    outgoing_links = adjacency_matrix.sum(axis=1)

    dangling_mask = (outgoing_links == 0)
    adjacency_matrix[dangling_mask] = np.ones(num_nodes)
    outgoing_links = adjacency_matrix.sum(axis=1)

    transition_probabilities = (adjacency_matrix.T / outgoing_links).T

    rank_values = np.full(num_nodes, 1 / num_nodes)

    for _ in range(max_iterations):
        updated_ranks = alpha * (transition_probabilities.T @ rank_values) + (1 - alpha) / num_nodes

        if np.linalg.norm(updated_ranks - rank_values, ord=1) < tolerance:
            break

        rank_values = updated_ranks

    ranked_nodes = sorted(enumerate(rank_values), key=lambda x: x[1], reverse=True)
    return ranked_nodes

%timeit compute_pagerank(matrix)
for alpha in alphas:
    ranks = compute_pagerank(matrix, alpha=alpha)
    ranks = [rank[0] for rank in ranks]
    print(f"Alpha: {alpha}, Ranks: {ranks[:20]}")

38.9 ms ± 14 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Alpha: 0.1, Ranks: [39, 860, 767, 371, 256, 926, 23, 295, 509, 329, 612, 692, 459, 564, 425, 742, 32, 197, 232, 160]
Alpha: 0.85, Ranks: [860, 767, 39, 926, 371, 742, 934, 256, 803, 32, 967, 944, 714, 719, 923, 720, 295, 329, 812, 197]
Alpha: 0.95, Ranks: [860, 767, 926, 39, 812, 866, 863, 865, 804, 805, 371, 742, 256, 934, 803, 967, 944, 32, 714, 719]


# np.linalg.eig implementation

In [ ]:
def compute_pagerank(adjacency_matrix, alpha=0.85):
    num_nodes = adjacency_matrix.shape[0]
    out_degrees = adjacency_matrix.sum(axis=1)

    dangling_nodes = (out_degrees == 0)
    adjacency_matrix[dangling_nodes] = np.ones(num_nodes)
    out_degrees = adjacency_matrix.sum(axis=1)

    transition_matrix = (adjacency_matrix.T / out_degrees).T

    google_matrix = alpha * transition_matrix + (1 - alpha) / num_nodes

    eigenvalues, eigenvectors = np.linalg.eig(google_matrix.T)
    dominant_eigenvector = np.abs(eigenvectors[:, np.argmax(eigenvalues.real)])

    rank_scores = dominant_eigenvector / np.sum(dominant_eigenvector)

    ranked_nodes = sorted(enumerate(rank_scores), key=lambda x: x[1], reverse=True)
    return ranked_nodes

%timeit compute_pagerank(matrix)
for alpha in alphas:
    ranks = compute_pagerank(matrix, alpha=alpha)
    ranks = [rank[0] for rank in ranks]
    print(f"Alpha: {alpha}, Ranks: {ranks[:20]}")

1.88 s ± 533 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Alpha: 0.1, Ranks: [39, 860, 767, 371, 256, 926, 23, 295, 509, 329, 612, 692, 459, 564, 425, 742, 32, 197, 232, 160]
Alpha: 0.85, Ranks: [860, 767, 39, 926, 371, 742, 934, 256, 803, 32, 967, 944, 714, 923, 719, 720, 295, 329, 812, 197]
Alpha: 0.95, Ranks: [860, 767, 926, 39, 812, 866, 863, 805, 804, 865, 371, 742, 256, 934, 803, 967, 944, 32, 714, 719]


# networkx implementation

In [ ]:
def pagerank(adj_matrix, alpha=0.85):
    G = nx.DiGraph(adj_matrix)
    ranks = nx.pagerank(G, alpha=alpha)
    sorted_ranks = sorted(ranks.items(), key=lambda item: item[1], reverse=True)
    return sorted_ranks

%timeit pagerank(matrix)
for alpha in alphas:
    ranks = pagerank(matrix, alpha=alpha)
    ranks = [rank[0] for rank in ranks]
    print(f"Alpha: {alpha}, Ranks: {ranks[:20]}")

117 ms ± 45.8 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Alpha: 0.1, Ranks: [39, 860, 767, 371, 256, 926, 23, 295, 509, 329, 612, 692, 459, 564, 425, 742, 32, 197, 232, 160]
Alpha: 0.85, Ranks: [860, 767, 39, 926, 371, 742, 934, 256, 803, 32, 967, 944, 714, 719, 923, 720, 295, 329, 812, 197]
Alpha: 0.95, Ranks: [860, 767, 926, 39, 812, 866, 863, 804, 805, 865, 371, 742, 934, 256, 803, 967, 944, 32, 714, 719]


# Question 2

In [ ]:
def compute_edge_rank(adj_matrix, page_rank_scores, damping=0.85):
    num_nodes = adj_matrix.shape[0]

    out_degrees = np.sum(adj_matrix, axis=1)

    edge_rank = {}
    for i in range(num_nodes):
        for j in range(num_nodes):
            if adj_matrix[i, j] == 1:
                if out_degrees[i] == 0:
                    continue
                contribution = damping * page_rank_scores[i] / out_degrees[i]
                edge_rank[(i, j)] = contribution

    return edge_rank

adj_matrix = np.zeros((11, 11))
#applying A roads
adj_matrix[2][3] = 1
adj_matrix[3][4] = 1
adj_matrix[4][5] = 1
#applying B roads
adj_matrix[9][8] = 1
adj_matrix[8][7] = 1
adj_matrix[7][6] = 1
#applying D roads
adj_matrix[0][3] = 1
adj_matrix[3][7] = 1
#applying E roads
adj_matrix[1][4] = 1
adj_matrix[10][8] = 1
adj_matrix[8][4] = 1
adj_matrix[4][8] = 1


ranks = pagerank(adj_matrix)
page_rank_scores = [x[1] for x in ranks]
compute_edge_rank(adj_matrix, page_rank_scores)

{(0, 3): 0.1455393860105159,
 (1, 4): 0.13953227237354224,
 (2, 3): 0.13169122103402006,
 (3, 4): 0.06054256978546994,
 (3, 7): 0.06054256978546994,
 (4, 5): 0.04531188974719507,
 (4, 8): 0.04531188974719507,
 (7, 6): 0.028769951439573335,
 (8, 4): 0.014384975719786667,
 (8, 7): 0.014384975719786667,
 (9, 8): 0.028769951439573335,
 (10, 8): 0.028769951439573335}